### TMA4320 - Project 2: Physics


# 0.0 Motivation

In [1]:
# Importing libraries
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Parameters
sigma_0 = 1000 # kg/m^2
sigma = 500 # kg/m^2
R = 10 # m, Radius
A_s = 0.5*np.pi*(R**2) # Cross-section of boat
g = 9.81 # m/s^2

## 1.? Tilstandsfunksjonen

For å kunne løse en høyere ordens differensialligning nummerisk, er det nyttig å bryte den ned til et system av førsteordens differensialligninger. Dette systemet beskriver tilstanden til det fysiske systemet.
I vårt tilfelle ønsker vi å beskrive rotasjonsvinkelen $\theta$ til skipet, om massesenteret $C$. Og vinkelhastigheten $\omega = \frac{d\theta}{dt}$.

Vi har følgende ligning: 
$$
    \sum \tau = I_C \frac{d\omega}{dt}
$$
der $\tau$ besriver driemomentet om massesenteret, og $I_C$ er skipets rotasjonstreghet.

Ved å introdusere vektoren $\vec{w} = ....$ ..

..


Kommer vi frem til 
$$
    f(t, \vec{w}) =   \begin{bmatrix}
                            \omega(t) \\
                            \sum \tau / I_C
                        \end{bmatrix}
$$

Vi må derfor definere alle dreiemomentfunksjonene, dette blir gjort i følgende celle:

In [ ]:
tau_B = lambda theta: -F_B*h*np.sin(theta)
tau_f = lambda theta: 
## ... gjør resten senere

## 1.3 Runge Kutta metoden

Implementerer fjerde ordens Runge Kutta.


In [1]:
def rk4_step(f, t, w, h):
    '''
    Tar inn en vektor-funksjon på formen f(t, _w_) = _y'_ = _y_ + _g(x)_, med nåværende tid (t) og 
    tilstand (w), og regner ut ett Runge Kutta steg.
    Inn:
        f: s-array, som beskriver et system av førsteordens diff-ligninger.
        t: float, nåvernde tid
        w: s-array, som beskriver systemet
    Ut:
        t_new: neste tidssteg (t+h)
        w_new: systemet etter ett beregnet tidssteg    
    '''
    k_1 = f(t, w)
    k_2 = f(t + 0.5*h, w + h*k_1*0.5)
    k_3 = f(t + 0.5*h, w + h*k_2*0.5)
    k_4 = f(t + 1.0*h, w + h*k_3)

    w_new = w + h*(1/6) * (k_1 + 2*k_2 + 2*k_3 + k_4)
    t_new = t + h

    return t_new, w_new




# X.0 Conclusion

Lorem Ipsum...